# Adding New Model Elements

This notebook walks through adding in-memory elements via PyMBE and tracking the additions to support updating model repositories after a session.

In [ ]:
import pymbe.api as pm
from pymbe.model import Element
from pymbe.model import MetaModel
from pymbe.model_modification import \
    create_new_classifier, create_new_relationship, create_new_feature

## Metamodel fields for new elements

Inspect the metamodels for attributes needed to make new elements

In [ ]:
mm = MetaModel()
mm.metamodel_hints['ActionDefinition'][0:3]

In [ ]:
mm.pre_made_dicts['ActionDefinition']

## New Package
The most basic element in a model for grouping things is the package. It holds a place within a modeling project to associate other model contents.

In [ ]:
package_model_data = {
    'name': "Power Pack Package",
    'isLibraryElement': False,
    'filterCondition': [],
    'ownedElement': [],
    'owner': {},
    '@type': "Package",
    '@id': str(uuid4())
}

In [ ]:
sysml_model = pm.Model(elements={})
sysml_model

In [ ]:
new_package = Element.new(data=package_model_data,model=sysml_model)
new_package

## New Classifier

For the KerML side of the metamodel, Classifier is one of the most basic kinds. We can build up a dictionary with owned attributes.

In [ ]:
classifier_model_data = {
    'name': "Battery Pack",
    'isLibraryElement': False,
    'documentation': [],
    'isConjugated': False,
    'isSufficient': False,
    'isAbstract': False,
    'ownedElement': [],
    'owner': {},
    '@type': "Classifier",
    '@id': str(uuid4())
}
new_classifier = Element.new(data=classifier_model_data,model=sysml_model)
new_classifier

## New Feature
A new Feature has some owned attributes and can be standalone in KerML.

## New OwningMembership

The OwningMembership, of which the OwningFeatureMembership is a sub-metatype, is what is used to organize the model hierarchy. Connecting two elements with this relationship should also updated derived fields such as owner and ownedElements.

In [ ]:
new_om_source = new_package
new_om_target = new_classifier
new_om_source_id = new_package._id
new_om_target_id = new_classifier._id
om_data = {
    'isLibraryElement': False,
    'documentation': [],
    'source': [{'@id': new_om_source_id}],
    'target': [{'@id': new_om_target_id}],
    'owningRelatedElement': {'@id': new_om_source_id},
    'ownedRelatedElement': [{'@id': new_om_target_id}],
    'relatedElement': [{'@id': new_om_source_id}, {'@id': new_om_target_id}],
    '@type': "OwningMembership",
    '@id': str(uuid4())
}
new_om = Element.new(data=om_data,model=sysml_model)
new_om
(new_om_source_id, new_om_target_id)


In [ ]:
new_om

In [ ]:
new_package.ownedElement

### Derive Properties Due to Relationship

On the two ends of the relationship are fields that need to have fields updated

In [ ]:
new_package._data.update(
    {'ownedElement': new_package._data['ownedElement'] + [{'@id': new_om_target_id}]
    }
)
new_package.resolve()

new_classifier._data.update(
    {'owner': {'@id': new_om_source_id}
    }
)
new_classifier.resolve()

In [ ]:
new_package._data

In [ ]:
new_package.ownedElement

In [ ]:
new_classifier.owner

In [ ]:
new_package['ownedElement']